In [15]:
from requests import get
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import csv
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
import re
from tqdm import tqdm
from dateutil.parser import parse
import os

In [18]:
winCount = 0
lossCount = 0
TRAIN_DIR = 'D:/PycharmProjects/marchMadness/train'
#print(year)
for year in range(2005,2019):
    TEAM_DIR = 'D:/PycharmProjects/marchMadness/teams/' + str(year) + '/'
    teamFiles = []
    teams = []
    for team in os.listdir(TEAM_DIR):
        #print(team)
        if team.split('.')[1] == 'csv':
            teamFiles.append(team)
            teams.append(team.split('.')[0])
    #print(teamFiles)

    for file in teamFiles:
        
        f = TEAM_DIR + file
        teamName = file
        rows = 0
                   
        with open(f) as csvfile:
            readCSV = csv.reader(csvfile, delimiter=',')
            cGame = []
            cDates = []
            cOpponent = []
            cTm = []
            cOpp = []
            cPdf = []
            cPf = []
            cPa = []
            for row in readCSV:
                if row[0] != 'G':
                    cGame.append(int(row[0]))
                    dt = parse(row[1])
                    cDates.append(dt)
                    cOpponent.append(row[2])
                    cTm.append(int(row[3]))
                    cOpp.append(int(row[4]))
                    cPdf.append((row[5]))
                    cPf.append((row[6]))
                    cPa.append((row[7]))
                    rows += 1

        cTeam = {'game': cGame,'date':cDates,'opponent': cOpponent,'tm': cTm,'opp': cOpp,'pdf': cPdf,'pf': cPf,'pa': cPa}
        #print(cTeam)

        for game in range(1,rows):
            opponent = cTeam['opponent'][game]
            opponent = ''.join([i for i in opponent if not i.isdigit()])
            opponent = opponent.rstrip()
            date = cTeam['date'][game]
            score = [cTeam['tm'][game],cTeam['opp'][game]]
            if score[0] > score[1]:
                label = 'win'
            else:
                label = 'loss'
            score = [str(cTeam['tm'][game]),str(cTeam['opp'][game])]
            cStats = [cTeam['pdf'][game-1],cTeam['pf'][game-1],cTeam['pa'][game-1]]
            #print(cStats)
            try:
                with open(TEAM_DIR + opponent + '.csv') as csvfile:
                    readCSV = csv.reader(csvfile, delimiter=',')
                    for row in readCSV:
                        if row[0] != 'G':
                            if parse(row[1]) == date:
                                oppGame = int(row[0])
                                oppGame -= 1
                                oppGame = str(oppGame)
                                with open(TEAM_DIR + opponent + '.csv') as oppfile:
                                    oppCSV = csv.reader(oppfile, delimiter=',')
                                    for oppRow in oppCSV:
                                        if oppRow[0] == oppGame:
                                            oppStats = [oppRow[5],oppRow[6],oppRow[7]]
            except FileNotFoundError:
                continue

                                    #print(oppStats)
            finalFile = [cStats,oppStats,score]
            #print(finalFile)
            if label == 'win':
                with open(TRAIN_DIR + '/' + label + '.' + str(winCount) + '.' + teamName.split('.')[0] + '.' + str(game+1)+ '.' + str(year) + '.csv', 'w') as csvFile:
                    writer = csv.writer(csvFile, lineterminator = '\n')
                    writer.writerows(finalFile)
                    winCount += 1
                csvFile.close()

            elif label == 'loss':
                with open(TRAIN_DIR + '/' + label + '.' + str(lossCount) + '.' + teamName.split('.')[0] + '.' + str(game+1)+ '.' + str(year) + '.csv', 'w') as csvFile:
                    writer = csv.writer(csvFile, lineterminator = '\n')
                    writer.writerows(finalFile)
                    lossCount += 1
                csvFile.close()

        print('Team #: {}/{}; Year: {}; Completion: {}%'.format(teamFiles.index(file)+1,len(teamFiles),year,round(((teamFiles.index(file)/len(teamFiles))*((year-2004)/14)),2)), end='\r')
